In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('cleaned_df.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2907 entries, 0 to 2906
Data columns (total 76 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   LotFrontage    2907 non-null   float64
 1   LotArea        2907 non-null   int64  
 2   OverallQual    2907 non-null   int64  
 3   OverallCond    2907 non-null   int64  
 4   YearBuilt      2907 non-null   int64  
 5   YearRemodAdd   2907 non-null   int64  
 6   MasVnrArea     2907 non-null   float64
 7   BsmtFinSF1     2907 non-null   float64
 8   BsmtFinSF2     2907 non-null   float64
 9   BsmtUnfSF      2907 non-null   float64
 10  TotalBsmtSF    2907 non-null   float64
 11  1stFlrSF       2907 non-null   int64  
 12  2ndFlrSF       2907 non-null   int64  
 13  LowQualFinSF   2907 non-null   int64  
 14  GrLivArea      2907 non-null   int64  
 15  BsmtFullBath   2907 non-null   float64
 16  BsmtHalfBath   2907 non-null   float64
 17  FullBath       2907 non-null   int64  
 18  HalfBath

In [4]:
df = pd.get_dummies(df, drop_first=True)

In [5]:
# Data to be predicted
df_to_predict = df[df['SalePrice'].isnull()]
df_to_predict = df_to_predict.drop(['SalePrice'], axis = 1)

In [6]:
df = df.dropna()

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1449 entries, 0 to 1448
Columns: 250 entries, LotFrontage to SaleCondition_Partial
dtypes: float64(12), int64(25), uint8(213)
memory usage: 731.6 KB


In [8]:
X = df.drop('SalePrice',axis=1)
y = df['SalePrice']

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

In [11]:
from sklearn.preprocessing import StandardScaler

In [12]:
scaler = StandardScaler()

In [13]:
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)

In [14]:
from sklearn.linear_model import ElasticNet

In [15]:
base_elastic_model = ElasticNet(max_iter=1000000)

In [16]:
param_grid = {'alpha':[5.5, 6, 6.5],
              'l1_ratio':[.98, .985, .987]}

In [17]:
from sklearn.model_selection import GridSearchCV

In [18]:
grid_model = GridSearchCV(estimator=base_elastic_model,
                          param_grid=param_grid,
                          scoring='neg_mean_squared_error',
                          cv=10,
                          verbose=2)

In [19]:
grid_model.fit(scaled_X_train,y_train)

Fitting 10 folds for each of 9 candidates, totalling 90 fits
[CV] END ...........................alpha=5.5, l1_ratio=0.98; total time=   0.0s
[CV] END ...........................alpha=5.5, l1_ratio=0.98; total time=   0.0s
[CV] END ...........................alpha=5.5, l1_ratio=0.98; total time=   0.0s
[CV] END ...........................alpha=5.5, l1_ratio=0.98; total time=   0.0s
[CV] END ...........................alpha=5.5, l1_ratio=0.98; total time=   0.0s
[CV] END ...........................alpha=5.5, l1_ratio=0.98; total time=   0.0s
[CV] END ...........................alpha=5.5, l1_ratio=0.98; total time=   0.0s
[CV] END ...........................alpha=5.5, l1_ratio=0.98; total time=   0.0s
[CV] END ...........................alpha=5.5, l1_ratio=0.98; total time=   0.0s
[CV] END ...........................alpha=5.5, l1_ratio=0.98; total time=   0.0s
[CV] END ..........................alpha=5.5, l1_ratio=0.985; total time=   0.0s
[CV] END ..........................alpha=5.5, l1

GridSearchCV(cv=10, estimator=ElasticNet(max_iter=1000000),
             param_grid={'alpha': [5.5, 6, 6.5],
                         'l1_ratio': [0.98, 0.985, 0.987]},
             scoring='neg_mean_squared_error', verbose=2)

In [20]:
grid_model.best_params_

{'alpha': 6, 'l1_ratio': 0.987}

In [21]:
y_pred = grid_model.predict(scaled_X_test)

In [22]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [23]:
mean_absolute_error(y_test,y_pred)

15680.936027026577

In [24]:
np.mean(df['SalePrice'])

180331.95514147688

In [25]:
mean_squared_error(y_test,y_pred)

542848515.7213072

In [26]:
np.sqrt(mean_squared_error(y_test,y_pred))

23299.109762420263